<a href="https://colab.research.google.com/github/aquapathos/BasicAI/blob/master/5CNN%20/CIFAR10PyTorch002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color=red>**【共有中に付き編集不可】**　改変したい場合はコピーを作成してください　PR,IAI</font>

In [ ]:
#@title 学生情報入力フォーム
#@markdown 以下のフォームに、学籍番号と氏名を入力してください。

#@markdown 例: 24AB1234山田 太郎, 24S1234情報 二郎
student_info = '23S0001山田太郎' #@param {type: "string"}

# 学籍番号と氏名の入力・抽出を行う関数
import re

# フォームに入力された文字列を直接使用
student_info_clean = student_info.strip()

# 学籍番号と氏名をキャプチャする正規表現
pattern = r'^\d{2,4}\s*[a-zA-Z]{1,2}\s*(\d{4})\s*(.+)$'

# 入力全体がパターンに完全に一致するかをチェック
match = re.fullmatch(pattern, student_info_clean)

# 学籍番号の確認を行う関数
def check_id_last_digits(student_id_full,need=2,mess="確認のため"):
    while True:
        try:
            check_id = input(f"{mess}学籍番号の下{need}桁を入力してください.")
            if check_id == student_id_full[-need:]:
                return True
            else:
                print(f"{check_id}{student_id_full[-need:]}入力が一致しません。\n")
                print("フォームのデータが間違っている場合は訂正してこのセルを再実行してください")
        except Exception:
            print("入力が不正です。")

# 下２桁だけチェック
def check2():
  check_id_last_digits(student_id_full,need=2,mess="次に進むには")

name, student_id_full = None, None
if match:
    # グループから学籍番号と氏名を取得
    student_id_full = match.group(1)
    name = match.group(2).strip()

    # 氏名が空ではないか確認
    if name:
        # 氏名に数字が含まれていないかチェック
        if any(char.isdigit() for char in name):
            print("フォームの学籍番号を確認し，間違いを修正して再実行してください。")
        else:
            # 学籍番号の確認
            check_pass = check_id_last_digits(student_id_full,need=4)
            if check_pass:
                print(f"ようこそ、{name}さん！　学習を始めましょう\n")
    else:
        print("氏名が入力されていません。再度フォームに入力し、このセルを再実行してください。")
else:
    print("入力形式が正しくありません。再度フォームに入力し、このセルを再実行してください。")

In [ ]:
# @title
# 動作環境の確認

import datetime
import pytz

def 時刻表示(phase):
    japan_tz = pytz.timezone('Asia/Tokyo')
    current_japan_time = datetime.datetime.now(japan_tz)
    formatted_time = current_japan_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
    print(phase,formatted_time)
    return formatted_time
時刻表示('開始時刻')

!pip -q install deep-translator requests
!pip -q install imagehash
!pip -q install icrawler deep-translator nltk torch transformers pillow

import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
if device.type != 'cpu':
  print(torch.cuda.get_device_name())
  print(torch.cuda.get_device_capability())

# 画像の収集
画像を認識するためには、まず事前にたくさんの事例をAIに機械学習させておく必要がありますが、
大量の画像を自分で用意するのは大変なので、プログラムで収集することにします。

以下、重要な指示は<font color='blue'>青字</font>，特に重要な指示は<font color='red'>赤字</font>にしてありますので、見落とさないように演習を進めていってください。

# 1. Google Drive をマウント
Google Colaboratory は一定時間経過すると作業内容が削除されてしまいます。消したくないデータや後日再利用したいデータは Google Drive に保存するようにプログラムすることで、残すことができます。


1. <font color='blue'>この説明の下のセルを実行する
2. アクセス許可の確認ダイアログが出る。「Googleドライブに接続」をクリック。
3. アカウントを選択する。
4. アクセスリクエストのダイアログに変わる。一番下までスクロールし「許可」をクリック。
5. 左のファイル一覧エリアで「更新アイコン」（回転マーク）をクリック（エリアが表示されていないなら「ファイルアイコン」（フォルダ型）をクリック）</font>


<img width="390" alt="googlecolab" src="https://user-images.githubusercontent.com/5820803/94802343-739cff00-0422-11eb-8c0d-affa919f8e58.png">

　<font color=blue>次のセルを実行してください</font>




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 2. 画像の収集と選別

自分で収集した画像を使って独自のデータセットを作り、先と同様の実験をやってみましょう。

## 2-1 画像の収集

画像を収集するために必要なライブラリを準備します。  <font color='blue'>次のセルを実行してください</font>

In [ ]:
#@title [C1.1] ライブラリ準備
# ==============================================================================
# 1. ライブラリ準備
# ==============================================================================
import os
import random
import shutil
import time
import requests
from pathlib import Path
import glob
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import nltk
from deep_translator import GoogleTranslator
from icrawler.builtin import BingImageCrawler
from PIL import Image
import imagehash
import numpy as np
import cv2
import torch

from transformers import CLIPModel, CLIPProcessor
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import wordnet

# from icrawler.builtin import GoogleImageCrawler
import warnings
import datetime
warnings.simplefilter('ignore')

ViewSIZE = 128 # 表示用画像サイズ高さと幅
# SIZE = 32 # モデルに入力する際の画像サイズ

機械学習の際に画像のサイズをすべて同じに揃えます。ViewSIZEは表示用の画像サイズ、SIZEは学習・認識時のサイズです。  
CIFAR10に合わせるためにSIZE = 32としています。

次に、収集する画像のカテゴリを決めましょう。



--
CIFAR-10 のカテゴリは
- airplane (飛行機)
- automobile (自動車）
- bird (鳥類）
- cat （ネコ)
- deer (シカ）
- dog （イヌ）
- frog （カエル）
- horse（ウマ）
- ship（船舶）
- truck（トラック）

でした。比較したいので、<font color=red>**ここに含まれるの中から「ネコ」とネコ以外のカテゴリを１つ**</font>、さらに、ここに<font color=red>**含まれないカテゴリを２つ**</font>，計４つのカテゴリを考えましょう。
以下では、CIFAR10のカテゴリから「ネコ」と「イヌ」、追加カテゴリとして「灯台」「スクータ」を選んだものとして説明していきます。

## **<font color=red>確認 実験に使う４つのカテゴリ</font>**
・「ネコ」を４つのうちの一つとして必ず含める．  
・CIFAR-10の10カテゴリから，ネコ以外に一つ選ぶ．  
・上に含まれないカテゴリを２つ，計４カテゴリを考える．

[2]のセルの myclasses のカテゴリ名を自分で選んだ４カテゴリに書き換えて実行してください。  

  <font color='blue'>下の２つのセルを順に実行してください</font>

In [ ]:
# @title [1.2]  カテゴリ条件のチェック関数
# ---------------------------------------------------------------
# myclasses の検証
# ---------------------------------------------------------------
CIFAR10_CATEGORIES = ['飛行機', '自動車', '鳥類', 'ネコ', 'シカ', 'イヌ', 'カエル', 'ウマ', '船舶', 'トラック']

# ---------------------------------------------------------------
# 検証用関数の定義 (汎用版)
# ---------------------------------------------------------------
def validate_myclasses(categories, required_category='ネコ', dataset_categories=CIFAR10_CATEGORIES, expected_size=4):
    """myclassesの条件を柔軟にチェック"""
    errors = []
    dataset_categories = list(dataset_categories) if dataset_categories else []

    if len(categories) != expected_size:
        errors.append(f"❌ カテゴリ数が{len(categories)}個です。{expected_size}個にしてください。")

    if required_category and required_category not in categories:
        errors.append(f"❌ 「{required_category}」が含まれていません。必ず含めてください。")

    dataset_except_required = [c for c in dataset_categories if c != required_category]
    dataset_in_myclasses = [c for c in categories if c in dataset_except_required]

    if len(dataset_in_myclasses) == 0:
        selectable = ', '.join(dataset_except_required) if dataset_except_required else '(データセット未指定)'
        errors.append(f"❌ データセットのカテゴリ（{required_category}以外）が1つも含まれていません。\n   次から1つ選んでください: {selectable}")
    elif len(dataset_in_myclasses) > 1:
        errors.append(f"❌ データセットのカテゴリ（{required_category}以外）が{len(dataset_in_myclasses)}個含まれています（{', '.join(dataset_in_myclasses)}）。\n   1個だけにしてください。")

    non_required_categories = [c for c in categories if c != required_category]
    non_dataset_categories = [c for c in non_required_categories if c not in dataset_categories]
    required_original_count = max(expected_size - 2, 0)

    if len(non_dataset_categories) < required_original_count:
        errors.append(
            f"❌ データセットに含まれない独自のカテゴリが{len(non_dataset_categories)}個しかありません。\n   {required_original_count}個必要です（データセット: {', '.join(dataset_categories)}）"
        )

    if errors:
        print("\n⚠️  以下の問題があります:\n")
        for error in errors:
            print(error)
        print("\n" + "="*60)
        print("🔧 修正例:")
        example_dataset = dataset_except_required[0] if dataset_except_required else 'データセットカテゴリ'
        example_originals = [f"独自カテゴリ{i+1}" for i in range(required_original_count)]
        example = [required_category if required_category else '必須カテゴリ', example_dataset] + example_originals
        example = example[:expected_size]
        print(f"myclasses = {example}")
        raise ValueError("myclasses の設定が条件を満たしていません。修正してください。")

    else:
        dataset_items = [c for c in categories if c in dataset_except_required]
        original_items = non_dataset_categories
        print("\n✅ すべての条件を満たしています！")
        if required_category:
            print(f"   - 必須: {required_category}")
        print(f"   - データセット: {dataset_items[0] if dataset_items else '(なし)'}")
        print(f"   - 独自カテゴリ: {', '.join(original_items) if original_items else '(なし)'}")
        print("="*60 + "\n")

print("上の説明を読まずに次を実行してしまわないようにするための関所です． \n上の指示通り「確認」記載通りにカテゴリを選び，下のセルのmyclassesを書き換えてから先に進んでくださいい．")
check2()

In [ ]:
#@title  [２]  設定 & 収集カテゴリ定義
# ==============================================================================
# 2. 設定 & AIクラス定義
# ==============================================================================
# 収集したいカテゴリ  （次の行は例です。'ネコ'以外を書き換えてください）
myclasses = ['ネコ','イヌ','灯台','スクータ']

# 保存先設定 (★変更: Driveではなくローカルに変更)
target_num = 300
TMPFOLDER = './tmp_images/'   # Colab内部の高速ストレージを使用

# 最終的な保存先 (Google Drive)
FINAL_SAVE_DIR = 'drive/MyDrive/tmp/'

# オプション設定
USE_CLIP_FILTER = True
CLIP_THRESHOLD = 0.22   # 0.20〜0.25推奨

# ---------------------------------------------------------------
# 検証実行
# ---------------------------------------------------------------
print("\n" + "="*60)
print("📋 myclasses の検証")
print("="*60)
print(f"設定されたカテゴリ: {myclasses}")
print(f"作業場所 (高速): {TMPFOLDER}")
print(f"保存場所 (Drive): {FINAL_SAVE_DIR}")

validate_myclasses(myclasses, required_category='ネコ', dataset_categories=CIFAR10_CATEGORIES, expected_size=4)


ネット上で公開されている画像を収集するためのライブラリ、icrawler を使って画像を収集します。 今回は Bingから画像を拾ってくることにします。（以前はGoogle検索を使っていましたが，仕様変更で使えなくなりました．）

次のセルは、上で選んだカテゴリの画像をそれぞれ３００枚を目標として収集し、一旦 Google Drive のマイドライブ内の 'tmp'という名のフォルダに保存するプログラムです。tmpフォルダ内にカテゴリごとのサブフォルダができます。（ "drive/MyDrive" が Google Driveのマイドライブを表しています）

  <font color='blue'>次のセルを実行してください</font> （定義だけなので何も起こりません）

In [ ]:
#@title 画像収集エージェントの定義
# ---------------------------------------------------------------
# 初期準備
# ---------------------------------------------------------------
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ 実行デバイス: {DEVICE}")

# ---------------------------------------------------------------
# ImageFilterAI クラス定義 (高速化+プログレスバー版)
# ---------------------------------------------------------------
# 顔検出器の準備
haar_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_path)

class ImageFilterAI:
    def __init__(self):
        # グローバルのDEVICE変数がなくても動くよう、ここでもチェック
        if torch.cuda.is_available():
            self.device = "cuda"
            self.batch_size = 24
            self.dtype = torch.float16
        else:
            self.device = "cpu"
            self.batch_size = 4
            self.dtype = torch.float32

        print(f"   🚀 AI Filter Device: {self.device}")

        self.model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(self.device, dtype=self.dtype)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.translator = GoogleTranslator(source='auto', target='en')
        self.face_cascade = face_cascade

    def _remove_duplicates(self, folder_path):
        """フォルダ内の重複画像をimagehashで検出・削除"""
        import imagehash
        from PIL import Image

        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
        if not image_files:
            return 0

        hashes = {}
        removed = 0

        for fname in tqdm(image_files, desc="Duplicate Check", unit="img", leave=False):
            path = os.path.join(folder_path, fname)
            try:
                with Image.open(path) as img:
                    hash_val = imagehash.phash(img)
                    if hash_val in hashes:
                        os.remove(path)
                        removed += 1
                    else:
                        hashes[hash_val] = fname
            except:
                continue

    def _has_human(self, image_path):
        try:
            img = cv2.imread(image_path)
            if img is None: return False
            height, width = img.shape[:2]
            if width > 1000:
                scale = 800 / width
                img = cv2.resize(img, None, fx=scale, fy=scale)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)
            return len(faces) > 0
        except Exception:
            return False

    def _prepare_labels(self, category_text, negative_words):
        pos = self.translator.translate(category_text)
        negs = [self.translator.translate(n) for n in negative_words]
        pos_labels = [f"{pos}", f"a {pos}", f"a real photo of a {pos}"]
        neg_labels = [f"a {n}" for n in negs] + ["text", "watermark", "illustration", "cartoon"]
        return pos_labels + neg_labels

    def filter(self, folder_path, category_text, negative_words, threshold, is_human_category=False):
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
        if not image_files: return

        labels = self._prepare_labels(category_text, negative_words)
        deleted = 0
        human_filtered = 0

        if not is_human_category:
            print(f"   👤 人物検出フィルタ適用中...")
            files_to_process = []
            for fname in tqdm(image_files, desc="Checking Faces", unit="img", leave=False):
                path = os.path.join(folder_path, fname)
                if self._has_human(path):
                    try: os.remove(path); human_filtered += 1
                    except: pass
                else:
                    files_to_process.append(fname)
            print(f"      ✅ 人物削除完了: {human_filtered} 枚削除しました")
            image_files = files_to_process

        if not image_files: return

        print(f"   🤖 CLIP意味フィルタ適用中...")
        batch_indices = range(0, len(image_files), self.batch_size)
        for i in tqdm(batch_indices, desc="AI Filtering", unit="batch", leave=False):
            batch_files = image_files[i:i+self.batch_size]
            images = []
            valid_batch_files = []
            for fname in batch_files:
                path = os.path.join(folder_path, fname)
                try:
                    with Image.open(path) as img:
                        images.append(img.convert("RGB"))
                        valid_batch_files.append(fname)
                except:
                    try: os.remove(path); deleted += 1
                    except: pass
            if not images: continue

            try:
                inputs = self.processor(text=labels, images=images, return_tensors="pt", padding=True).to(self.device)
                if self.device == "cuda":
                    inputs["pixel_values"] = inputs["pixel_values"].to(dtype=self.dtype)
                with torch.no_grad():
                    outputs = self.model(**inputs)
                    probs = outputs.logits_per_image.softmax(dim=1)
                pos_count = len(labels) // 2
                for fname, p in zip(valid_batch_files, probs):
                    path = os.path.join(folder_path, fname)
                    pos_score = p[:pos_count].max().item()
                    best_idx = p.argmax().item()
                    if best_idx >= pos_count or pos_score < threshold:
                        try: os.remove(path); deleted += 1
                        except: pass
            except Exception:
                continue
        # 重複除去
        print(f"   🔍 重複画像検出中...")
        duplicates = self._remove_duplicates(folder_path)
        if duplicates is None:
            duplicates = 0
        if duplicates > 0:
            print(f"      ✅ 重複削除完了: {duplicates} 枚削除")

        print(f"      ✅ CLIP選別完了: {deleted} 枚削除（合計削除: {deleted + human_filtered + duplicates} 枚）")

時刻表示('CLIPモデル読み込み完了')
# ---------------------------------------------------------------
# UI/インタラクション部分は直接セルで input() を使う前提
# - search_keyword_ui(), process_global_queue() は削除済み
# ---------------------------------------------------------------

# GLOBAL_SEARCH_QUEUE = {}
print("✅ 準備完了。次のセルで UI や収集処理を実行できます")

# [3] 検索語の決定

検索したいものを見つけるためのキーワードと排除したいものを表すキーワードを決めます。

## 検索キーワードの修正
検索は英語を使った方が多くヒットするので、まず日本語を英語に変換し、類義語をAIに提案させます。

語のニュアンスが日本語と英語とでずれている場合があります。例えば、「スクータ」は英語では「scooter」ですが、英語の「scooter」には海上スクータが含まれます。

スクータ　→　scooter　→ sea scooter, motor scooter, water scooter

AIが検索語の候補を提案してきます。不要な語があれば削除して上書きしてください。

→　スクータ, scooter, motor scooter

そのまま採用すれば良い時は空欄のままEnterを押してください。

## 検索キーワードの追加
AIの提案に検索語を加えたい場合は、+に続けて付け加えたい語を並べてください。

+ホンダスクータ, Yamaha scooter.
 →　スクータ, scooter, motor scooter, ホンダスクータ, Yamaha scooter

## 排除キーワード

検索画像には望ましくないものもたくさん含まれますので、これは含んでほしくないというものを聞いてきますので答えてください。

デフォルトでイラストやテキストは除外リストとなるので、+に続けて排除キーワードを並べて答えてください。

例）+猫バス、キャットウォーク

In [ ]:
# @title
# ---------------------------------------------------------------
# StrategyPlanner (動的ネガティブリスト + 文脈認識)
# ---------------------------------------------------------------
# 必要なモデル準備
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=torch.device('cpu')) # 軽量なのでCPUでOK
translator = GoogleTranslator(source='auto', target='en')
ACTIONS_AND_CONTEXTS = ['photo', 'close up', 'side view', 'on white background', 'in context', 'in use']
HUMAN_KEYWORDS = ['person', 'people', 'human', 'man', 'woman', 'child', 'baby', 'kid','boy', 'girl', 'adult', 'teenager', 'infant', 'toddler','人', '人間', '男性', '女性', '子供', '赤ちゃん', '大人', '少年', '少女']

class StrategyPlanner:
    def __init__(self, similarity_threshold=0.6, max_candidates=40):
        self.to_en = translator
        self.similarity_threshold = similarity_threshold
        self.max_candidates = max_candidates

        # 1. 【常に】除外したいもの
        self.basic_negatives = [
            'toy', 'cartoon', 'illustration', 'sketch', 'drawing', 'vector',
            'logo', 'text', 'font', 'watermark', 'symbol', 'icon', 'clipart',
            'screenshot', '3d render', 'origami', 'miniature', 'model',
            'plastic', 'sticker', 'decals'
        ]

        # 2. 【食べ物以外の場合のみ】除外したいもの
        self.food_negatives = [
            'food', 'dish', 'recipe', 'cooking', 'meal', 'ingredient', 'cuisine'
        ]

    def _clip_text_sim(self, base, cands):
        if not cands: return []
        base_emb = embedder.encode(base, convert_to_tensor=True)
        cand_embs = embedder.encode(cands, convert_to_tensor=True)
        sims = util.cos_sim(base_emb, cand_embs)[0].cpu().numpy()
        return sims.tolist()

    def _is_human_category(self, category_jp, category_en):
        cat_jp_lower = category_jp.lower()
        cat_en_lower = category_en.lower()
        for keyword in HUMAN_KEYWORDS:
            if keyword in cat_jp_lower or keyword in cat_en_lower:
                return True
        return False

    def _is_food_category(self, category_en):
        # Aチーム: 食べ物
        food_anchors = ["food", "dish", "meal", "cuisine", "beverage", "dessert"]
        # Bチーム: それ以外
        non_food_anchors = [
            "animal", "mammal", "bird", "fish", "insect", "pet",
            "vehicle", "car", "bicycle", "boat", "aircraft",
            "architecture", "building", "place", "scenery",
            "technology", "tool", "device", "furniture", "clothing"
        ]

        food_score = max(self._clip_text_sim(category_en, food_anchors) + [0])
        non_food_score = max(self._clip_text_sim(category_en, non_food_anchors) + [0])
        print(f"   ⚖️ カテゴリ判定: Food({food_score:.2f}) vs Non-Food({non_food_score:.2f})")

        if (food_score > non_food_score) and (food_score > 0.3):
            return True
        return False

    def _is_bad_word(self, word, active_negatives):
        padded_word = f" {word.lower()} "
        for bad in active_negatives:
            if f" {bad} " in padded_word:
                return True
        return False

    def _fetch_from_conceptnet(self, word_en):
        cands = set()
        url = f"http://api.conceptnet.io/query?rel=/r/IsA&end=/c/en/{word_en}&limit=100"
        try:
            time.sleep(1.0)
            response = requests.get(url, timeout=3.0)
            if response.status_code == 200:
                data = response.json()
                for edge in data['edges']:
                    start_label = edge['start']['label'].lower()
                    if edge['start']['language'] != 'en' or len(start_label.split()) > 3:
                        continue
                    cands.add(start_label)
        except Exception:
            pass
        return list(cands)

    def _get_recursive_hyponyms(self, synset, depth=0, max_depth=2):
        results = set()
        if depth >= max_depth: return results
        for hypo in synset.hyponyms():
            for lemma in hypo.lemmas():
                name = lemma.name().replace('_', ' ').lower()
                results.add(name)
            results.update(self._get_recursive_hyponyms(hypo, depth + 1, max_depth))
        for inst in synset.instance_hyponyms():
            for lemma in inst.lemmas():
                name = lemma.name().replace('_', ' ').lower()
                results.add(name)
        return results

    def _fetch_from_wordnet(self, word_en):
        cands = set()
        synsets = wordnet.synsets(word_en, pos=wordnet.NOUN)
        for syn in synsets[:3]:
            found_words = self._get_recursive_hyponyms(syn, depth=0, max_depth=2)
            cands.update(found_words)
        return list(cands)

    def get_subcategories(self, category_jp):
        # 1. 翻訳
        try:
            cat_en = self.to_en.translate(category_jp).lower()
        except:
            cat_en = category_jp

        print(f"🔍 '{category_jp}' ({cat_en}) の関連語を収集中...")

        is_human = self._is_human_category(category_jp, cat_en)
        is_food = self._is_food_category(cat_en)

        # 動的ネガティブリストの構築
        active_negatives = self.basic_negatives.copy()
        if not is_food:
            active_negatives.extend(self.food_negatives)
            print("   🛡️ 非食品モード: food, dish などを除外します")
        else:
            print("   🍽️ 食品モード: food, dish などを許容します")

        # 2. 候補収集
        candidates = self._fetch_from_conceptnet(cat_en)
        wn_candidates = self._fetch_from_wordnet(cat_en)
        candidates.extend(wn_candidates)

        candidates = list(set(candidates))
        if cat_en not in candidates: candidates.append(cat_en)

        # 3. フィルタリング
        THRESHOLD_SCORE = 0.25
        print(f"   🤖 {len(candidates)} 個の候補をAI評価中 (閾値: {THRESHOLD_SCORE})...")

        sims = self._clip_text_sim(cat_en, candidates)

        valid_candidates = []
        for word, score in zip(candidates, sims):
            if word == cat_en:
                valid_candidates.append((word, 1.0))
                continue
            if score < THRESHOLD_SCORE:
                continue
            if self._is_bad_word(word, active_negatives):
                continue
            valid_candidates.append((word, score))

        valid_candidates.sort(key=lambda x: x[1], reverse=True)
        final_list = [w for w, s in valid_candidates[:self.max_candidates]]

        if cat_en in final_list: final_list.remove(cat_en)
        final_list.insert(0, cat_en)
        if category_jp in final_list: final_list.remove(category_jp)
        final_list.insert(0, category_jp)

        final_list = list(dict.fromkeys(final_list))

        print(f"   -> 採用: {final_list[:10]} ... 計 {len(final_list)} 個")
        return final_list, cat_en, is_human

# ---------------------------------------------------------
# インタラクション関数（改良版: 番号指定で削除可能な方式）
# ---------------------------------------------------------
def safe_input_loop(prompt_text, default_value, is_additive_mode=False):
    """
    ユーザーフレンドリーな入力ループ

    Args:
        prompt_text: 説明文
        default_value: AI提案のデフォルト値（リストまたは文字列）
        is_additive_mode: Trueの場合、入力は常にデフォルトに追加される（排除語用）
    """
    while True:
        print("-" * 40)
        print(prompt_text)

        if isinstance(default_value, list):
            current_list = default_value.copy()
        else:
            current_list = [default_value]

        # 番号付きリストで表示
        print(f"👉 AI提案:")
        for idx, word in enumerate(current_list, 1):
            print(f"   {idx}. {word}")

        if is_additive_mode:
            print("\n   操作:")
            print("   - そのままEnterで提案を採用")
            print("   - 追加したい語を入力（カンマ区切り）")
            user_in = input(">>> ").strip()

            if not user_in:
                actual_val = current_list
                mode_msg = "【AI提案をそのまま採用】"
            else:
                add_words = [w.strip().lstrip('+').lstrip('＋') for w in user_in.replace('、', ',').replace('，', ',').split(',') if w.strip()]
                actual_val = current_list + add_words
                actual_val = list(dict.fromkeys(actual_val))
                mode_msg = f"【AI提案に追加】 (+ {','.join(add_words)})"
        else:
            print("\n   操作:")
            print("   - そのままEnterで提案を採用")
            print("   - 削除したい番号を入力（例: 2,4,5）")
            print("   - 追加したい語を入力（例: +柴犬,チワワ）")
            print("   - 全て書き換え（例: =柴犬,チワワ,秋田犬）")
            user_in = input(">>> ").strip()

            if not user_in:
                actual_val = current_list
                mode_msg = "【AI提案をそのまま採用】"
            elif user_in.startswith('-'):
                # 削除モード: -2,4,5 形式
                try:
                    nums_str = user_in[1:].replace(' ', '')
                    nums = [int(n) for n in nums_str.split(',') if n]
                    actual_val = [w for i, w in enumerate(current_list, 1) if i not in nums]
                    mode_msg = f"【番号 {','.join(map(str, nums))} を削除】"
                except:
                    print("⚠️ 番号の指定が不正です。再入力してください。")
                    continue
            elif user_in.startswith('+') or user_in.startswith('＋'):
                # 追加モード（全角＋にも対応）
                start_char = user_in[0]
                add_words = [w.strip().lstrip('+').lstrip('＋') for w in user_in[1:].replace('、', ',').replace('，', ',').split(',') if w.strip()]
                actual_val = current_list + add_words
                actual_val = list(dict.fromkeys(actual_val))
                mode_msg = f"【AI提案に追加】 (+ {','.join(add_words)})"
            elif user_in.startswith('=') or user_in.startswith('＝'):
                # 全置換モード（全角＝にも対応）
                actual_val = [w.strip().lstrip('+').lstrip('＋') for w in user_in[1:].replace('、', ',').replace('，', ',').split(',') if w.strip()]
                mode_msg = "【全て書き換え】"
            elif all(c.isdigit() or c in ',、， ' for c in user_in):
                # 番号のみ入力された場合は削除として扱う
                try:
                    nums = [int(n) for n in user_in.replace('、', ',').replace('，', ',').replace(' ', '').split(',') if n]
                    actual_val = [w for i, w in enumerate(current_list, 1) if i not in nums]
                    mode_msg = f"【番号 {','.join(map(str, nums))} を削除】"
                except:
                    print("⚠️ 入力が不正です。再入力してください。")
                    continue
            else:
                # その他は全置換として扱う
                actual_val = [w.strip().lstrip('+').lstrip('＋') for w in user_in.replace('、', ',').replace('，', ',').split(',') if w.strip()]
                mode_msg = "【入力内容で置換】"

        disp_val = ",".join(actual_val) if actual_val else "(空)"

        print(f"\n📝 {mode_msg}")
        print(f"   確定されるキーワード: 【 {disp_val} 】")
        confirm = input("   これで確定しますか？ (Enter = OK / n = やり直し): ").strip().lower()
        if confirm == '' or confirm == 'y':
            return actual_val
        else:
            print("🔄 再入力します...\n")

def remove_duplicates(lst):
    return list(dict.fromkeys(lst))

# ---------------------------------------------------------
# メイン実行処理
# ---------------------------------------------------------
時刻表示('検索語選択の開始')
planner = StrategyPlanner()
GLOBAL_SEARCH_QUEUE = {}

print("\n" + "="*60)
print("📝 検索キーワード計画 (AI除外提案モード)")
print("="*60 + "\n")

temp_plans = []

for categ in myclasses:
    print(f"\n📂 カテゴリ: 【 {categ} 】 の設定")

    sub_cats, cat_en, is_human = planner.get_subcategories(categ)
    default_keywords = sub_cats[:8]

    print(f"\n👇 '{categ}' の検索キーワードを確定してください")
    prompt_kwd = "入力例:"
    base_keywords = safe_input_loop(prompt_kwd, default_keywords)

    # 除外ワード提案
    rec_negs = ['illustration', 'cartoon', 'toy', 'sketch', 'clipart']
    if not planner._is_food_category(cat_en):
        rec_negs.extend(['food', 'recipe'])

    print(f"\n👇 '{categ}' の除外ワード(NOT)を確定してください")
    neg_words = safe_input_loop("AIがカテゴリに合わせて推奨リストを作成しました:", rec_negs)
    if isinstance(neg_words, str): neg_words = [neg_words]

    final_queries = []
    final_queries.extend(base_keywords)
    for base in base_keywords:
        for action in ACTIONS_AND_CONTEXTS:
            final_queries.append(f"{base} {action}")
    final_queries = remove_duplicates(final_queries)

    temp_plans.append({
        'categ': categ,
        'queries': final_queries,
        'neg': neg_words,
        'is_human': is_human,
        'cat_en': cat_en
    })

    print(f"✅ '{categ}' の設定完了 (クエリ数: {len(final_queries)})")

for plan in temp_plans:
    GLOBAL_SEARCH_QUEUE[plan['categ']] = {
        'queries': plan['queries'],
        'neg': plan['neg'],
        'is_human': plan['is_human'],
        'cat_en': plan['cat_en']
    }

print("\n✨ 全カテゴリの設定が完了しました！ 次のセルへ進んでください。")
時刻表示('検索語決定完了')

# [4] 自動収集と自動選別
Bing検索で指定されたキーワードで画像を集め、画像を認識してキーワードに合っていないものや排除対象が写っているものを取り除きます。

In [ ]:
#@title [４] 自動収集と自動選別（変数活用・パイプライン版）
# ==============================================================================
# 4. フェーズ2：全自動実行（収集 ＋ CLIP選別 ＋ 即時転送）
# ==============================================================================
import shutil
import os
import time
from icrawler.builtin import BingImageCrawler

時刻表示('収集開始')

if not GLOBAL_SEARCH_QUEUE:
    print("⚠️ エラー: STEP 3 がまだ実行されていません。")
else:
    print("\n" + "="*60)
    print("🚀 画像収集スタート (パイプラインモード)")
    print(f"   作業場所: {TMPFOLDER}")
    print(f"   保存場所: {FINAL_SAVE_DIR}")
    print("   1カテゴリ終わるごとにGoogle Driveへ転送します。")
    print("="*60 + "\n")

    clip_filter = ImageFilterAI() if USE_CLIP_FILTER else None

    # 作業フォルダ（ローカル）の初期化
    if os.path.exists(TMPFOLDER):
        shutil.rmtree(TMPFOLDER)

    # 追加のCLIP除外ワード
    CLIP_EXTRA_NEGS = [
        'text', 'watermark', 'logo', 'vector', 'low quality', 'blurry',
        'screenshot', 'collage', 'frame', 'border', 'cgi', '3d render'
    ]

    for i, (categ, plan) in enumerate(GLOBAL_SEARCH_QUEUE.items()):
        print(f"\n📂 カテゴリ {i+1}/{len(GLOBAL_SEARCH_QUEUE)} 処理中: 【 {categ} 】")

        queries = plan['queries']
        neg_list = plan['neg']
        is_human = plan['is_human']
        neg_str = " ".join([f"-{n}" for n in neg_list])

        # ローカルの一時パス (./tmp_images/ネコ など)
        local_cat_dir = os.path.join(TMPFOLDER, categ)
        os.makedirs(local_cat_dir, exist_ok=True)

        # クローラー設定 (ローカルに保存)
        crawler = BingImageCrawler(storage={"root_dir": local_cat_dir}, downloader_threads=10)

        # --- 1. 収集 ---
        query_index = 0
        while True:
            current_count = len(os.listdir(local_cat_dir))

            if current_count >= target_num * 1.3:
                print(f"   🛑 収集完了 (現在: {current_count}枚)")
                break

            if query_index >= len(queries):
                print("   ⚠️ クエリ切れ")
                break

            q_word = queries[query_index]
            full_query = f"{q_word} {categ} {neg_str}"

            if query_index % 5 == 0:
                print(f"      [{current_count}枚] 🔍 {full_query}")

            try:
                crawler.crawl(keyword=full_query, filters={'type': 'photo'}, max_num=50, overwrite=False, file_idx_offset='auto')
            except Exception:
                pass

            query_index += 1
            time.sleep(0.5)

        # --- 2. 選別 (CLIP + 人物) ---
        if clip_filter:
            combined_negs = neg_list + CLIP_EXTRA_NEGS
            clip_filter.filter(local_cat_dir, categ, combined_negs, CLIP_THRESHOLD, is_human_category=is_human)

        final_count = len(os.listdir(local_cat_dir))
        print(f"   🎉 選別完了: {final_count} 枚")

        # --- 3. Google Driveへ即時転送 ---
        print(f"   🚚 Google Driveへ転送中... ({categ})")

        # 最終保存先のパス (drive/MyDrive/tmp/ネコ など)
        drive_cat_dir = os.path.join(FINAL_SAVE_DIR, categ)

        os.makedirs(drive_cat_dir, exist_ok=True)

        # ファイルをコピー
        files_to_move = os.listdir(local_cat_dir)
        for fname in files_to_move:
            src = os.path.join(local_cat_dir, fname)
            dst = os.path.join(drive_cat_dir, fname)
            shutil.copy2(src, dst)

        print(f"   ✅ 転送完了！ Driveのフォルダを確認してください: {drive_cat_dir}")
        print(f"   (次のカテゴリに進みます...)")

    print("\n" + "="*60)
    print("🏁 全ての処理が完了しました")
    print("="*60)

時刻表示('収集完了')

ERRORが出るのはリンク切れ(404)やアクセス制限(403)であり、プログラムのエラーではないので気にしなくても構いません。

## 2-2 画像の選別

![Google Drive](https://raw.githubusercontent.com/aquapathos/pictures/gh-pages/BasicAIFig001.png)　　　　　　 ![Google Drive](https://raw.githubusercontent.com/aquapathos/pictures/gh-pages/BasicAIFig002.png)   

![Google Drive](https://github.com/aquapathos/pictures/blob/gh-pages/BasicAIFig003.png?raw=true)

1. <font color='blue'>ブラウザで新しいタブをつくり、[ドライブ](https://drive.google.com/) を開きます。
2. **マイドライブ**の中の**tmp**フォルダを開いてみてください。 自分が決めたカテゴリのフォルダが並んでいるはずです。
3.「ネコ」を例として説明します。**ネコ** フォルダを開きます。
4. リストレイアウトからグリッドレイアウトに切り替えて不適だと思われる画像を削除してください。
5. 同様にして、ほかのカテゴリ（例ではイヌ、灯台、スクータ）についても不要画像を削除してください。</font>

不適なのは、次のような画像です。
- ネコが写っていない画像。写っていても小さすぎる画像
- 主要被写体がネコではない画像、つまりネコ以外のものが目立っている画像
- 文字が目立つ画像。できれば文字は入っていないことが望ましいですが、目立たなければOK
- デフォルメされたぬいぐるみやイラスト。
- GIF 動画
- 同一の写真

パッと見て、これは「ネコ」が主題の写真だと感じられるものを残して、そうでないもの、他のモノが多くの面積を占めているものは削除してください。(厳密なルールはありません。あなた自身がこれは何かと問われて「ネコ」と答える写真を選べばいいです。あなたの選んだものが教師データとなります。)

# 2-3　画像のリサイズ

何度も使うので関数を定義しておきます。  <font color='blue'>次のセルを実行してください。</font>　　（コードは非表示になっていますが、実行する必要があります。）

In [ ]:
#@title リサイズ関数の定義
def resize(foldername, size=ViewSIZE):
    imgnames = glob.glob(foldername+"/*") # 画像ファイル名のリスト
    images = []
    for imgname in imgnames:
        img = cv2.cvtColor(np.array(Image.open(imgname).convert('RGBA')),cv2.COLOR_RGBA2RGB)
        height = img.shape[0]
        width = img.shape[1]
        if height > width :
            m = (height - width)//2
            img = img[m:m+width]
        else:
            m = (width - height)//2
            img = img[:,m:m+height]
        img = cv2.resize(img, (size,size))
        images.append(img)
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        cv2.imwrite(imgname,img)
    return images

さっそくリサイズしてみましょう。　　<font color='blue'>次のセルを実行してください。</font>




In [ ]:
# @title
input('不適切画像の削除は終了したら Enterを押してください')
# リサイズを実行
images = []
for i,categ in enumerate(myclasses):
  images.append(resize(FINAL_SAVE_DIR+categ))
  print(f"{categ} - {len(images[i])} 枚の画像がリサイズされました")
images = np.array(images, dtype=object) ## numpy 仕様変更あり 2024

表示用関数を定義し、表示して確認してみます。　　<font color='blue'>次のセルを実行してください。</font>　コードは非表示にしてあります。

In [ ]:
#@title 表示用関数の定義
import math
# start番からnpic枚表示する関数を定義
plt.rcParams['figure.figsize'] = (12.0, 7.0)
def showimg(images, start = 0, npic = 48):
    n = npic if len(images) >= start+npic else len(images) - start
    plt.figure(figsize=(8,7.5*(math.ceil(n/8))/6),dpi=150)
    i = 0
    while True:
        if i < n :
            plt.subplot((n-1)//8+1,8,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(images[start+i])
            plt.title("{}".format(start+i))
            i += 1
        else:
            break

さっそく表示させてみましょう。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
# @title
for i in range(len(images)):
  showimg(images = images[i],start=0,npic=8) # 各カテゴリ16枚ずつ表示

## 2-4 学習用画像データの保存と読み込み
python にはメモリ上に置かれた変数の値をそっくりそのままファイルとして保存する関数が用意されていますので、その関数を使って変数の中身を Google ドライブに保存したり、読み出したりできるようにしておきます。　　<font color='blue'>次のセルを実行してください。</font>


In [ ]:
#@title
import pickle
import os

# 画像データを pickle 形式で保存
def storeCategoryImages(cat, fname, folder = "."):
    os.makedirs(folder, exist_ok=True)
    f = open(folder+"/"+fname,'wb')
    pickle.dump(cat,f)
    f.close

# pickle 形式で保存された画像データの読み込み
def loadCategoryImages(fname, folder = "."):
    f = open(folder+"/"+fname,'rb')
    cat = pickle.load(f)
    f.close
    return cat

さっそくリサイズした画像をpickle 形式で保存しましょう。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
GFOLDER = "drive/MyDrive/CIFAR10"  # データ保存用のフォルダ名
for i,categ in enumerate(myclasses):
  print(i,categ)
  storeCategoryImages(images[i],f"{categ}.pkl",folder=GFOLDER)

今度は逆に pickle 形式で保存されたデータを変数に読み込み、正しく復元できるか確認しておきましょう。一つ目のカテゴリだけ試してみます。　　<font color='blue'>次のセルを実行してください。</font>

In [ ]:
testimg = loadCategoryImages("ネコ.pkl", folder=GFOLDER)
showimg(images = testimg,start = 4,npic=8)

# CIFAR10PyTorch003 に進んでください。

------